## Basic Usage: Predict Patient Risk from WSI and clinical data

In [4]:
from utils.utils import read_yaml
from utils.model_factory import load_model
import torch

cfg = read_yaml('projects/configs/cfg_progpath.yaml')
model = load_model(cfg)
model.load_state_dict(torch.load('weights/progpath.pt'), strict=True)


loading ProgPath model


<All keys matched successfully>

In [ ]:
import pandas as pd
import random
import torch
from datasets.SurvivalBagDataset import process_clinical

def encode_cancer_type(cancer_type):
    cancer_types = ['blca', 'brca', 'cesc', 'crc', 'gbm', 'hnsc', 'rcc', 'lgg', 'lihc', 'luad', 'lusc', 'paad', 'skcm', 'stad', 'ucec']
    encoding = [0] * len(cancer_types)
    if cancer_type in cancer_types:
        encoding[cancer_types.index(cancer_type)] = 1
    return encoding

random.seed(1)
patient_csv = pd.read_csv('csvs/sr_crc.csv')

patient_data = random.sample(list(patient_csv['patient_id']), 1)
patient_data = patient_csv[patient_csv['patient_id'] == patient_data[0]]
patch_features = torch.load(f'features/{patient_data["filename"].values[0]}')
clinical_feature = process_clinical(patient_data, columns=['age', 'sex', 'stage'])['processed_df'].drop(columns=['filename']).values
clinical_feature = torch.from_numpy(clinical_feature).float()
cancer_type = 'crc'
cancer_encoding = torch.tensor(encode_cancer_type(cancer_type)).float()

In [6]:
model.eval()
model.to('cuda')
patch_features = patch_features.to('cuda')
res = model(h=patch_features, clinical_feature=clinical_feature.to('cuda'), cancer_encoding=cancer_encoding.to('cuda'))
risk = res['bag_logits'][0][1:]
print('patient id:', patient_data['patient_id'].values[0], 'risk:', risk.item())

patient id: 98 risk: -0.8011984825134277


## Evaluation

In [7]:
!python3 eval.py --config_path projects/configs/cfg_progpath.yaml

loading ProgPath model
loading test set clinical csvs
loading 1 external val dataloaders
sr_crc cindex_now: 0.7765380443204711 pvalue: 2.496161383249256e-10
